In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import load_dataset

ابتدا دیتاست را لود کرده و ستون‌های جمله و لیبل جمله را می‌خوانیم.

In [33]:
dataset = load_dataset("AmazonScience/massive", "fa-IR")

sentences_train = dataset['train']['utt']
labels_train = dataset['train']['scenario']

sentences_validation = dataset['validation']['utt']
labels_validation = dataset['validation']['scenario']

sentences_test = dataset['test']['utt']
labels_test = dataset['test']['scenario']


Found cached dataset massive (C:/Users/ASUS/.cache/huggingface/datasets/AmazonScience___massive/fa-IR/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
vectorizer = TfidfVectorizer()
sentences_train_tfidf = vectorizer.fit_transform(sentences_train)
sentences_validation_tfidf = vectorizer.transform(sentences_validation)
sentences_test_tfidf = vectorizer.transform(sentences_test)

In [36]:
model = LogisticRegression(max_iter=1000, solver='sag')
model.fit(sentences_train_tfidf, labels_train)

LogisticRegression(max_iter=1000, solver='sag')

In [37]:
labels_map = ['social',
 'transport',
 'calendar',
 'play',
 'news',
 'datetime',
 'recommendation',
 'email',
 'iot',
 'general',
 'audio',
 'lists',
 'qa',
 'cooking',
 'takeaway',
 'music',
 'alarm',
 'weather']

محاسبه دقت مدل براساس داده‌های آموزش، تست و ارزیابی دیتاست اصلی:

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('معیارهای ارزیابی براساس داده‌های آموزش:')
# پیش‌بینی برچسب‌ها برای داده‌های آموزش
train_predictions = model.predict(sentences_train_tfidf)

# محاسبه دقت (Accuracy)
accuracy = accuracy_score(labels_train, train_predictions)
print("Accuracy:", accuracy)

# محاسبه دقت (Precision)
precision = precision_score(labels_train, train_predictions, average='macro')
print("Precision:", precision)

# محاسبه بازخوانی (Recall)
recall = recall_score(labels_train, train_predictions, average='macro')
print("Recall:", recall)

# محاسبه اندازه F1
f1 = f1_score(labels_train, train_predictions, average='macro')
print("F1:", f1)


print('معیارهای ارزیابی براساس داده‌های ولیدیشن:')
# پیش‌بینی برچسب‌ها برای داده‌های آموزش
validation_predictions = model.predict(sentences_validation_tfidf)

# محاسبه دقت (Accuracy)
accuracy = accuracy_score(labels_validation, validation_predictions)
print("Accuracy:", accuracy)

# محاسبه دقت (Precision)
precision = precision_score(labels_validation, validation_predictions, average='macro')
print("Precision:", precision)

# محاسبه بازخوانی (Recall)
recall = recall_score(labels_validation, validation_predictions, average='macro')
print("Recall:", recall)

# محاسبه اندازه F1
f1 = f1_score(labels_validation, validation_predictions, average='macro')
print("F1:", f1)


print('معیارهای ارزیابی براساس داده‌های تست:')
# پیش‌بینی برچسب‌ها برای داده‌های آموزش
test_predictions = model.predict(sentences_test_tfidf)

# محاسبه دقت (Accuracy)
accuracy = accuracy_score(labels_test, test_predictions)
print("Accuracy:", accuracy)

# محاسبه دقت (Precision)
precision = precision_score(labels_test, test_predictions, average='macro')
print("Precision:", precision)

# محاسبه بازخوانی (Recall)
recall = recall_score(labels_test, test_predictions, average='macro')
print("Recall:", recall)

# محاسبه اندازه F1
f1 = f1_score(labels_test, test_predictions, average='macro')
print("F1:", f1)

معیارهای ارزیابی براساس داده‌های آموزش:
Accuracy: 0.9307799200972728
Precision: 0.9496857631963886
Recall: 0.9120210407905804
F1: 0.9290359751362901
معیارهای ارزیابی براساس داده‌های ولیدیشن:
Accuracy: 0.8726020659124447
Precision: 0.8976197670616699
Recall: 0.8442572660422971
F1: 0.8672443345096942
معیارهای ارزیابی براساس داده‌های تست:
Accuracy: 0.8685272360457297
Precision: 0.903860340866525
Recall: 0.8334936937306404
F1: 0.8628816100160324


تست مدل روی جملات کاملا جدید که عملکرد نسبتا خوبی دارد

In [44]:
new_sentences = [
    'نیم ساعت میخوابم بیدارم کن',
    'روسیه به اوکراین حمله کرد.',
    'خورش مرغ کبابی چطوری بپزم؟',
    'هوای قم گرمتر از اهوازه',
    'فردا سه شنبه است یا جمعه؟'
]
new_sentences_tfidf = vectorizer.transform(new_sentences)
new_predictions = model.predict(new_sentences_tfidf)
print("جملات جدید:")
for sentence, prediction in zip(new_sentences, new_predictions):
    print("{}\n قصد: {}".format(sentence, labels_map[prediction]))


جملات جدید:
نیم ساعت میخوابم بیدارم کن
 قصد: alarm
روسیه به اوکراین حمله کرد.
 قصد: qa
خورش مرغ کبابی چطوری بپزم؟
 قصد: cooking
هوای قم گرمتر از اهوازه
 قصد: weather
فردا سه شنبه است یا جمعه؟
 قصد: calendar


همانطور که مشاهده می‌شود مدل LogisticRegression که یک مدل پایه غیر ترنسفورمری است در رقابت و مقایسه با مدل‌های ترنسفورمری و در کنار روش naive bayse عملکرد نسبتا خوبی دارد و درصدهای معیارهای ارزیابی آن تقریبا برابر است. تفاوت‌های جزئی در بعضی عملکردها وجود دارد و برخی تستها طبیعتا نتیجه متفاوت در مدل‌های مختلف دارد. 